In [5]:
import os
import pandas as pd
import requests
import calendar
import json
from matplotlib import pyplot as plt

import numpy as np
import seaborn as sns

from consts import *
import utils

In [6]:
estimated_production_by_roof = utils.flatten_json(DATA_DIR.joinpath("city-attributes.json"), MONTHS_ABBR_UNITS)

In [7]:
estimated_production_by_roof

,etak_id,roof_id,annual_kwh,monthly_average_kwh,roof_area,azimuth,tilt,orientation,total_roof_area,lat,...,Mar_kwh,Apr_kwh,May_kwh,Jun_kwh,Jul_kwh,Aug_kwh,Sep_kwh,Oct_kwh,Nov_kwh,Dec_kwh
0,7106760,1,5242.83,436.90,26.198233,-96.837100,4.921,none,26.20,58.345708,...,488.21,624.31,754.57,721.51,722.32,634.87,477.82,265.07,97.54,69.20
1,728449,2,6839.15,569.93,34.221977,94.493128,7.901,none,34.22,58.354539,...,624.75,821.54,994.65,945.24,950.62,829.34,621.95,350.34,127.04,85.65
2,737756,3,4339.29,361.61,23.309351,-43.279136,31.058,south,204.24,58.357790,...,372.15,518.12,672.03,656.47,648.22,543.84,377.81,195.39,63.29,40.99
3,737756,4,2319.89,193.32,19.183207,135.375705,31.538,north,204.24,58.357790,...,144.31,272.28,404.20,439.15,421.09,309.36,170.86,69.82,19.44,8.09
4,737756,5,8657.23,721.44,71.746268,-134.736848,30.623,east,204.24,58.357790,...,559.64,1001.78,1523.67,1627.52,1548.66,1148.61,646.63,265.17,72.95,30.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79438,724636,79439,915.09,76.26,6.919663,126.304497,27.131,west,747.13,58.362351,...,59.64,109.53,158.33,168.97,162.75,121.58,69.50,28.85,7.69,3.02
79439,724636,79440,873.90,72.83,6.602727,126.236821,27.069,west,747.13,58.362351,...,56.95,104.60,151.20,161.33,155.41,116.11,66.39,27.56,7.35,2.88
79440,724636,79441,867.44,72.29,6.548483,126.098284,27.052,west,747.13,58.362351,...,56.57,103.84,150.06,160.08,154.22,115.25,65.93,27.38,7.29,2.85
79441,711101,79442,2023.34,168.61,10.122619,96.483074,8.288,none,33.79,58.365328,...,184.84,243.04,294.23,279.60,281.20,245.33,184.00,103.67,37.61,25.37


In [8]:
estimated_production_by_roof[ETAK_ID].unique().size

20884

In [9]:
estimated_production_by_roof.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79443 entries, 0 to 79442
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   etak_id              79443 non-null  object 
 1   roof_id              79443 non-null  int64  
 2   annual_kwh           79443 non-null  float64
 3   monthly_average_kwh  79443 non-null  float64
 4   roof_area            79443 non-null  float64
 5   azimuth              79443 non-null  float64
 6   tilt                 79443 non-null  float64
 7   orientation          79443 non-null  object 
 8   total_roof_area      79443 non-null  float64
 9   lat                  79443 non-null  float64
 10  lon                  79443 non-null  float64
 11  points               79443 non-null  object 
 12  Jan_kwh              79443 non-null  float64
 13  Feb_kwh              79443 non-null  float64
 14  Mar_kwh              79443 non-null  float64
 15  Apr_kwh              79443 non-null 

In [10]:
calculate_pv_area_vect = np.vectorize(utils.calculate_pv_area, excluded=["roof_area_pv_ratio"])
estimated_production_by_roof[PV_AREA_M2] = calculate_pv_area_vect(
    estimated_production_by_roof[ROOF_AREA],
    ROOF_AREA_PV_RATIO
)

calculate_annual_kwh_pv_m2_vect = np.vectorize(utils.calculate_annual_kwh_pv_m2)
estimated_production_by_roof[ANNUAL_KWH_PV_M2] = calculate_annual_kwh_pv_m2_vect(
    estimated_production_by_roof[ANNUAL_KWH],
    estimated_production_by_roof[PV_AREA_M2]
)

estimated_production_by_roof[MONTHS_ABBR_UNITS_PV_M2] = estimated_production_by_roof[MONTHS_ABBR_UNITS]\
    .divide(estimated_production_by_roof[PV_AREA_M2], axis=0)

In [11]:
estimated_production_by_roof

,etak_id,roof_id,annual_kwh,monthly_average_kwh,roof_area,azimuth,tilt,orientation,total_roof_area,lat,...,Mar_kwh/pv_m2,Apr_kwh/pv_m2,May_kwh/pv_m2,Jun_kwh/pv_m2,Jul_kwh/pv_m2,Aug_kwh/pv_m2,Sep_kwh/pv_m2,Oct_kwh/pv_m2,Nov_kwh/pv_m2,Dec_kwh/pv_m2
0,7106760,1,5242.83,436.90,26.198233,-96.837100,4.921,none,26.20,58.345708,...,20.705807,26.478037,32.002583,30.600453,30.634806,26.925905,20.265150,11.242065,4.136835,2.934888
1,728449,2,6839.15,569.93,34.221977,94.493128,7.901,none,34.22,58.354539,...,20.284236,26.673568,32.294063,30.689831,30.864507,26.926817,20.193327,11.374757,4.124705,2.780864
2,737756,3,4339.29,361.61,23.309351,-43.279136,31.058,south,204.24,58.357790,...,17.739661,24.697765,32.034354,31.292639,30.899378,25.923787,18.009463,9.313859,3.016910,1.953913
3,737756,4,2319.89,193.32,19.183207,135.375705,31.538,north,204.24,58.357790,...,8.358584,15.770738,23.411680,25.436020,24.389967,17.918450,9.896387,4.044046,1.125985,0.468581
4,737756,5,8657.23,721.44,71.746268,-134.736848,30.623,east,204.24,58.357790,...,8.666963,15.514241,23.596581,25.204873,23.983593,17.788149,10.014148,4.106601,1.129753,0.472653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79438,724636,79439,915.09,76.26,6.919663,126.304497,27.131,west,747.13,58.362351,...,9.576574,17.587561,25.423523,27.132020,26.133256,19.522465,11.159823,4.632531,1.234806,0.484930
79439,724636,79440,873.90,72.83,6.602727,126.236821,27.069,west,747.13,58.362351,...,9.583582,17.602155,25.444033,27.148715,26.152494,19.539065,11.172152,4.637814,1.236863,0.484648
79440,724636,79441,867.44,72.29,6.548483,126.098284,27.052,west,747.13,58.362351,...,9.598491,17.619009,25.461368,27.161508,26.167215,19.554996,11.186645,4.645690,1.236928,0.483573
79441,711101,79442,2023.34,168.61,10.122619,96.483074,8.288,none,33.79,58.365328,...,20.288996,26.677330,32.296210,30.690345,30.865969,26.928692,20.196794,11.379356,4.128269,2.784743


In [7]:
estimated_production_by_roof.to_csv(ROOFS_PATH, index=False)